<a href="https://colab.research.google.com/github/mint-aguccim/Machine_Learning_Programming/blob/main/%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91_%EB%B0%8F_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%A0%80%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# [코인 트랙] [1] 비트코인 차트 이미지 생성 (yfinance)
# ==========================================

# Colab에서 한 번만 실행할 셀 (라이브러리 설치)
!pip install --upgrade yfinance mplfinance pandas tqdm

# ---------------------------------------------------------
# 비트코인 차트 이미지 생성 (최종 완성본)
# ---------------------------------------------------------
import yfinance as yf
import mplfinance as mpf
import pandas as pd
import matplotlib.pyplot as plt # [추가] 메모리 정리를 위해 필요
import os
from google.colab import drive
from tqdm.notebook import tqdm  # 진행바

# 1. 구글 드라이브 마운트
try:
    drive.mount('/content/drive', force_remount=False)
except Exception as e:
    print("드라이브 마운트 경고 (이미 되어있으면 무시):", e)

# 2. 경로 설정
project_path = '/content/drive/MyDrive/P실무프로젝트_3조'
save_path = os.path.join(project_path, 'bitcoin_images')
os.makedirs(save_path, exist_ok=True)
print(f"저장 경로: {save_path}")

# 3. 데이터 다운로드
ticker = 'BTC-USD'
# 학습 데이터 확보를 위해 기간을 넉넉히 잡는 것이 좋습니다.
start_date = '2018-01-01'
end_date   = '2024-05-20'

print("비트코인 데이터 다운로드 중...")
try:
    df_coin = yf.download(ticker, start=start_date, end=end_date, progress=False)
except Exception as e:
    print("데이터 다운로드 실패:", e)
    raise

if df_coin is None or len(df_coin) == 0:
    raise RuntimeError("다운로드된 데이터가 없습니다. ticker 또는 기간을 확인하세요.")

# 4. 데이터 전처리
# MultiIndex 컬럼 정리 (yfinance 호환성)
if isinstance(df_coin.columns, pd.MultiIndex):
    df_coin.columns = df_coin.columns.get_level_values(0)

# DatetimeIndex 보장
if not isinstance(df_coin.index, pd.DatetimeIndex):
    df_coin.index = pd.to_datetime(df_coin.index)

# 결측치 제거
df_coin = df_coin.dropna(how='any')
print(f"데이터 준비 완료: 총 {len(df_coin)}일치")

# 5. 차트 생성 루프
mc = mpf.make_marketcolors(up='r', down='b', inherit=True)
mpf_style = mpf.make_mpf_style(marketcolors=mc)

window_size = 20
count = 0
N = len(df_coin) - window_size

print("차트 이미지 변환 시작...")

# tqdm 진행바 사용
for i in tqdm(range(N), desc='이미지 생성'):
    subset = df_coin.iloc[i : i + window_size]
    target_date = subset.index[-1].strftime('%Y-%m-%d')
    filename = os.path.join(save_path, f"{target_date}.png")

    # 이미 있으면 건너뛰기
    if os.path.exists(filename):
        continue

    try:
        mpf.plot(
            subset,
            type='candle',
            style=mpf_style,
            axisoff=True,
            volume=False,
            # dpi=50 정도면 CNN 학습용(약 200~300px)으로 적당합니다.
            savefig=dict(fname=filename, dpi=50, pad_inches=0)
        )

        # [★핵심 추가] 메모리에 쌓인 그래프 찌꺼기 삭제
        plt.close('all')

        count += 1
    except Exception as e:
        print(f"[WARN] 이미지 생성 실패 ({target_date}): {e}")
        continue

print(f"\n완료! 총 {count}장의 비트코인 차트가 저장되었습니다.")
print(f"저장 위치: {save_path}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 28.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
Mounted at /content/drive
저장 경로: /content/drive/MyDrive/P실무프로젝트_3조/bitcoin_images
비트코인 데이터 다운로드 중...


/tmp/ipython-input-1977993175.py:39: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_coin = yf.download(ticker, start=start_date, end=end_date, progress=False)


데이터 준비 완료: 총 2331일치
차트 이미지 변환 시작...


이미지 생성:   0%|          | 0/2311 [00:00<?, ?it/s]


완료! 총 0장의 비트코인 차트가 저장되었습니다.
저장 위치: /content/drive/MyDrive/P실무프로젝트_3조/bitcoin_images
